In [ ]:
# !pip install earthengine-api

In [ ]:
!earthengine authenticate

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
ee.Initialize()

In [ ]:
# HELPER FUNCTIONS

#--------------Function to identify and apply cloud masks--------------------
def cloud_masking(image):
  cloud_qa = image.select('QA_PIXEL')

  # Extract 2-bit confidence values
  cloud_conf = cloud_qa.rightShift(8).bitwiseAnd(0b11)  # Bits 8-9
  shadow_conf = cloud_qa.rightShift(10).bitwiseAnd(0b11)  # Bits 10-11
  snow_conf = cloud_qa.rightShift(12).bitwiseAnd(0b11)  # Bits 12-13
  cirrus_conf = cloud_qa.rightShift(14).bitwiseAnd(0b11)  # Bits 14-15

  # Mask for low confidence (0 = None, 1 = Low)
  cloud_conf_mask = cloud_conf.lte(1)
  shadow_conf_mask = shadow_conf.lte(1)
  snow_conf_mask = snow_conf.lte(1)
  cirrus_conf_mask = cirrus_conf.lte(1)

  # combine masks
  final_mask = cloud_conf_mask.And(shadow_conf_mask).And(snow_conf_mask).And(cirrus_conf_mask)

  # Apply the mask to the image
  masked_image = image.updateMask(final_mask)

  return masked_image


# -----------Function to extract the band value at the point-----------------
def extract_band_value(image,point,band_name):

    # Apply cloud masking function
    image = cloud_masking(image)

    # Count valid pixels
    valid_pixel_count = image.select(band_name).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=point,
        scale=30
    ).get(band_name)

    # calculate average temperature of the masked pixels
    band_value = image.select(band_name).reduceRegion(
        reducer=ee.Reducer.mean(),  # mean value over the region
        geometry=point,
        scale=30
    ).get(band_name)

    # If no valid pixels, set the band value to a negative placeholder
    band_value = ee.Algorithms.If(valid_pixel_count, band_value, -999999)

    # Add the extracted band value and date as properties
    return image.set('date', image.date().format('yyyy-MM-dd')).set('ST_Value', band_value)


# ---------Function to apply multiple arguments to a .map operation------------
def apply_band_extraction(collection, point, band_name, scale, offset):

    # Apply the function to each image in the collection using the map function
    collection_with_values = collection.map(lambda img: extract_band_value(img, point, band_name))

    # Number of images in the collection
    count = collection_with_values.size()
    print(f"Number of images in the collection: {count.getInfo()}")

    # Extract the time series data by aggregate data
    band_values = collection_with_values.aggregate_array('ST_Value').getInfo()

    # Apply scale and offset to convert DN to Kelvin
    bands = [(value * scale)+offset for value in band_values]

    # Extract corresponding dates
    dates = collection_with_values.aggregate_array('date').getInfo()

    # remove entries with only cloud cover
    filtered_dates, filtered_bands = zip(*[
        (date, value) for date, value in zip(dates, bands) if value > 0
    ])

    # Convert back to lists
    dates_list = list(filtered_dates)
    band_values = list(filtered_bands)

    return band_values, dates_list

In [ ]:
# --------------------------LANDSAT 4----------------------------------------

# Define the point of interest (longitude, latitude)
lake_point = ee.Geometry.Point([-73.69789, 43.43342]).buffer(200)  # 200 meter buffer

# Extract the bounding box (bounds) of the buffer
bbox = lake_point.bounds()

# Convert the bounding box to a polygon geometry
lake_bbox = ee.Geometry.Polygon(bbox.getInfo()['coordinates'])

# Load Landsat 4
landsat4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2") \
  .filterDate('1982-01-01', '2025-01-01') \
  .filterBounds(lake_bbox)\

# Kelvin conversion factors (Landsat 4)
L4_scale = 0.00341802
L4_offset = 149

# Use custom functions to extract date and temperature data
band_values, dates_list = apply_band_extraction(landsat4, lake_bbox, 'ST_B6', L4_scale, L4_offset)

# Print the time series values
print("Landsat 4 Time Series:")
for date, value in zip(dates_list, band_values):
    print(f"{date}: {value} [K]")


# -------------Write data to CSV file-----------------

# Define satellite name
satellite_name = "Landsat 4"

# Create a DataFrame
df = pd.DataFrame({
    "Date": dates_list,
    "Temperature (K)": band_values,
    "Satellite": [satellite_name] * len(dates_list)
})

# Save as CSV
csv_filename = "landsat_temperature_timeseries.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file saved: {csv_filename}")



In [ ]:
# --------------------------LANDSAT 5----------------------------------------

# Define the point of interest (latitude, longitude)
lake_point = ee.Geometry.Point([-73.69789, 43.43342]).buffer(200)  # 200 meter buffer

# Extract the bounding box (bounds) of the buffer
bbox = lake_point.bounds()

# Convert the bounding box to a polygon geometry
lake_bbox = ee.Geometry.Polygon(bbox.getInfo()['coordinates'])

# Load Landsat 5
landsat5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") \
  .filterDate('1982-01-01', '2025-01-01') \
  .filterBounds(lake_bbox)\

# Kelvin conversion factors (Landsat 4)
L5_scale = 0.00341802
L5_offset = 149

# Use custom functions to extract date and temperature data
band_values, dates_list = apply_band_extraction(landsat5, lake_bbox, 'ST_B6', L5_scale, L5_offset)

# Print the time series values
print("Landsat 5 Time Series:")
for date, value in zip(dates_list, band_values):
    print(f"{date}: {value} [K]")


# -------------Write data to CSV file-----------------

# Define satellite name
satellite_name = "Landsat 5"

# Create a DataFrame
df_new = pd.DataFrame({
    "Date": dates_list,
    "Temperature (K)": band_values,
    "Satellite": [satellite_name] * len(dates_list)
})

# Append to existing CSV
csv_filename = "landsat_temperature_timeseries.csv"
df_new.to_csv(csv_filename, mode="a", header=False, index=False)

print(f"Appended data to: {csv_filename}")

In [ ]:
# --------------------------LANDSAT 7----------------------------------------

# Define the point of interest (latitude, longitude)
lake_point = ee.Geometry.Point([-73.69789, 43.43342]).buffer(200)  # 200 meter buffer

# Extract the bounding box (bounds) of the buffer
bbox = lake_point.bounds()

# Convert the bounding box to a polygon geometry
lake_bbox = ee.Geometry.Polygon(bbox.getInfo()['coordinates'])

# Load Landsat 7
landsat7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") \
  .filterDate('1982-01-01', '2025-01-01') \
  .filterBounds(lake_bbox)\

# Kelvin conversion factors (Landsat 4)
L7_scale = 0.00341802
L7_offset = 149

# Use custom functions to extract date and temperature data
band_values, dates_list = apply_band_extraction(landsat7, lake_bbox, 'ST_B6', L7_scale, L7_offset)

# Print the time series values
print("Landsat 7 Time Series:")
for date, value in zip(dates_list, band_values):
    print(f"{date}: {value} [K]")


# -------------Write data to CSV file-----------------

# Define satellite name
satellite_name = "Landsat 7"

# Create a DataFrame
df_new = pd.DataFrame({
    "Date": dates_list,
    "Temperature (K)": band_values,
    "Satellite": [satellite_name] * len(dates_list)
})

# Append to existing CSV
csv_filename = "landsat_temperature_timeseries.csv"
df_new.to_csv(csv_filename, mode="a", header=False, index=False)

print(f"Appended data to: {csv_filename}")

In [ ]:
# --------------------------LANDSAT 8----------------------------------------

# Define the point of interest (latitude, longitude)
lake_point = ee.Geometry.Point([-73.69789, 43.43342]).buffer(200)  # 200 meter buffer

# Extract the bounding box (bounds) of the buffer
bbox = lake_point.bounds()

# Convert the bounding box to a polygon geometry
lake_bbox = ee.Geometry.Polygon(bbox.getInfo()['coordinates'])

# Load Landsat 8
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
  .filterDate('1982-01-01', '2025-01-01') \
  .filterBounds(lake_bbox)\

# Kelvin conversion factors (Landsat 8)
L8_scale = 0.00341802
L8_offset = 149

# Use custom functions to extract date and temperature data
band_values, dates_list = apply_band_extraction(landsat8, lake_bbox, 'ST_B10', L8_scale, L8_offset)

# Print the time series values
print("Landsat 8 Time Series:")
for date, value in zip(dates_list, band_values):
    print(f"{date}: {value} [K]")


# -------------Write data to CSV file-----------------

# Define satellite name
satellite_name = "Landsat 8"

# Create a DataFrame
df_new = pd.DataFrame({
    "Date": dates_list,
    "Temperature (K)": band_values,
    "Satellite": [satellite_name] * len(dates_list)
})

# Append to existing CSV
csv_filename = "landsat_temperature_timeseries.csv"
df_new.to_csv(csv_filename, mode="a", header=False, index=False)

print(f"Appended data to: {csv_filename}")

In [ ]:
# --------------------------LANDSAT 9----------------------------------------

# Define the point of interest (latitude, longitude)
lake_point = ee.Geometry.Point([-73.69789, 43.43342]).buffer(200)  # 200 meter buffer

# Extract the bounding box (bounds) of the buffer
bbox = lake_point.bounds()

# Convert the bounding box to a polygon geometry
lake_bbox = ee.Geometry.Polygon(bbox.getInfo()['coordinates'])

# Load Landsat 9
landsat9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2") \
  .filterDate('1982-01-01', '2025-01-01') \
  .filterBounds(lake_bbox)\

# Kelvin conversion factors (Landsat 9)
L9_scale = 0.00341802
L9_offset = 149

# Use custom functions to extract date and temperature data
band_values, dates_list = apply_band_extraction(landsat9, lake_bbox, 'ST_B10', L9_scale, L9_offset)

# Print the time series values
print("Landsat 9 Time Series:")
for date, value in zip(dates_list, band_values):
    print(f"{date}: {value} [K]")


# -------------Write data to CSV file-----------------

# Define satellite name
satellite_name = "Landsat 9"

# Create a DataFrame
df_new = pd.DataFrame({
    "Date": dates_list,
    "Temperature (K)": band_values,
    "Satellite": [satellite_name] * len(dates_list)
})

# Append to existing CSV
csv_filename = "landsat_temperature_timeseries.csv"
df_new.to_csv(csv_filename, mode="a", header=False, index=False)

print(f"Appended data to: {csv_filename}")

In [ ]:
#----------SORT THE CSV FILE BY DATE----------------

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("landsat_temperature_timeseries.csv")

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort the DataFrame by the 'Date' column
df_sorted = df.sort_values(by='Date')

# Save the sorted DataFrame back to a CSV
df_sorted.to_csv("landsat_temperature_timeseries.csv", index=False)

print("CSV sorted and saved as landsat_temperature_timeseries.csv")


In [ ]:
#--------------PREPARE DATA FOR PLOTTING-------------------

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("landsat_temperature_timeseries.csv")

# Convert columns to python lists
dates_list = df['Date'].tolist()
band_values = df['Temperature (K)'].tolist()
satellite = df['Satellite'].tolist()

#--------------CALCULATE YEARLY AVERAGE---------------------

# Find average yearly temps
df['Date'] = pd.to_datetime(df['Date'])

# Extract the year
df['Year'] = df['Date'].dt.year

# Group by year and calculate the average temperature
yearly_averages = df.groupby('Year')['Temperature (K)'].mean()

# Get separate lists for years and averages
years = yearly_averages.index.tolist()  # Get years as a list
averages = yearly_averages.values.tolist()  # Get averages as a list

# Fit a linear regression trendline
slope, intercept = np.polyfit(np.array(years), np.array(averages), 1)



In [ ]:
# PLOT THE TEMPS AND DATES
step = 25
plt.plot(dates_list, band_values)
plt.xlabel('Date')
plt.xticks(dates_list[::step], rotation=45, ha='right')
plt.ylabel('Temperature [K]')
plt.title('Lake George NY: Temperature Time Series')
plt.show()

# PLOT THE YEARLY AVERAGES
step = 2
plt.plot(years, averages,label='Yearly Averages')
plt.plot(years, slope * np.array(years) + intercept, label='Fitted Line')
plt.xlabel('Year')
plt.xticks(years[::step], rotation=45, ha='right')
plt.ylabel('Temperature [K]')
plt.title('Lake George NY: Yearly Avg. Temperature')
plt.legend()
plt.show()

In [ ]:
## CLOUD MASKING

# Load Landsat 4
image = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2") \
  .filterDate('1984-01-01', '2025-01-01') \
  .filterBounds(lake_bbox)\
  .toList(2).get(0)  # Get the first image by converting to a list

image = ee.Image(image)  # Convert back to an ee.Image

cloud_qa = image.select('QA_PIXEL')

# Extract 2-bit confidence values
cloud_conf = cloud_qa.rightShift(8).bitwiseAnd(0b11)  # Bits 8-9
shadow_conf = cloud_qa.rightShift(10).bitwiseAnd(0b11)  # Bits 10-11
snow_conf = cloud_qa.rightShift(12).bitwiseAnd(0b11)  # Bits 12-13
cirrus_conf = cloud_qa.rightShift(14).bitwiseAnd(0b11)  # Bits 14-15

# Mask for low confidence (0 = None, 1 = Low)
cloud_conf_mask = cloud_conf.lte(1)
shadow_conf_mask = shadow_conf.lte(1)
snow_conf_mask = snow_conf.lte(1)
cirrus_conf_mask = cirrus_conf.lte(1)

# combine masks
final_mask = cloud_conf_mask.And(shadow_conf_mask).And(snow_conf_mask).And(cirrus_conf_mask)

# Apply the mask to the image
masked_image = image.updateMask(final_mask)



In [ ]:
# VISUALIZE CLOUD MASKING ON A MAP


# Apply scale and offset to convert to reflectance
def apply_scale_offset(image):
    scale_factor = 0.0000275
    offset = -0.2
    bands = ['SR_B1', 'SR_B2', 'SR_B3']

    return image.select(bands).multiply(scale_factor).add(offset)

# Scale the image
scaled_image = apply_scale_offset(image)

# Set visualization parameters for reflectance
vis_params = {
    'bands': ['SR_B1', 'SR_B2', 'SR_B3'],  # Red, Green, Blue
    'min': 0,
    'max': 0.3,  # 0.3 for improved contrast
    'gamma': 1.4  # adjust contrast
}

# Visualize the cloud mask as black and transparent on everything else
mask_visual = final_mask.Not().selfMask()  # invert the mask and set 1 = black
mask_vis_params = {"min": 0, "max": 1, "palette": ["black"]}

# Create a map
my_map = geemap.Map()

# Add the scaled image
my_map.addLayer(scaled_image, vis_params, "Landsat 4 Reflectance")

# add the cloud mask image
my_map.addLayer(mask_visual, mask_vis_params, "Cloud Mask")

# bounding box layer
my_map.addLayer(lake_bbox, {'color': 'red'}, "Bounding Box")


# Display the map
my_map